# <p style="text-align: center;, font-style: strong;">Partie 2 : MNIST with Multiple Layer Perceptron (MLP)</p>

### <p style="text-align: center;">(Almond 0.8.0, Scala 2.12.8)</p>


## Dependencies

In [1]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("darwin-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [2]:
import org.platanios.tensorflow.api.ops.{Files, Image => TImage}
import java.nio.file.{ Files => JFiles, Paths, Path }
import java.awt.{Image => JImage}
import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}
import scala.io.Source
import scala.collection.mutable.ListBuffer
import java.awt.image.BufferedImage
import java.awt.Color
import java.awt.image.BufferedImage
import javax.imageio.ImageIO
import scala.util.Random
import java.util.Base64
import java.io.PrintWriter
import scala.io.Source
import java.io.ByteArrayInputStream
import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.client.{Session, FeedMap}
import org.platanios.tensorflow.api.core.types.DataType
import org.platanios.tensorflow.data.image.MNISTLoader
import java.nio.file.{Files, Paths}
import org.platanios.tensorflow.api.core.{Shape, types}
import scala.language.postfixOps
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.implicits.helpers.{OutputStructure, OutputToDataType, OutputToShape}
import org.platanios.tensorflow.api.learn.ClipGradientsByGlobalNorm
import org.platanios.tensorflow.api.ops.Output
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.learn.layers.{Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean, ScalarSummary}
import org.platanios.tensorflow.api.learn.{Model, StopCriteria, Configuration}
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.api.learn.hooks.{SummarySaver, StepHookTrigger, CheckpointSaver}
import org.platanios.tensorflow.api.config.TensorBoardConfig
import java.util.Base64
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.implicits.helpers.{OutputStructure, OutputToDataType, OutputToShape}
import org.platanios.tensorflow.api.learn.ClipGradientsByGlobalNorm
import org.platanios.tensorflow.api.ops.Output
import org.platanios.tensorflow.data.image.MNISTLoader
import java.nio.file.Paths

import org.platanios.tensorflow.api.ops.{Files, Image => TImage}

import java.nio.file.{ Files => JFiles, Paths, Path }

import java.awt.{Image => JImage}

import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}

import scala.io.Source

import scala.collection.mutable.ListBuffer

import java.awt.image.BufferedImage

import java.awt.Color

import java.awt.image.BufferedImage

import javax.imageio.ImageIO

import scala.util.Random

import java.util.Base64

import java.io.PrintWriter

import scala.io.Source

import java.io.ByteArrayInputStream

import org.platanios.tensorflow.api.tf

import org.platanios.tensorflow.api.tensors.Tensor

import org.platanios.tensorflow.api.core.Shape

import org.platanios.tensorflow.api.core.client.{Session, FeedMap}

import org.platanios.tensorflow.api.core.types.DataType

import org.platanios.tensorflow.data.image.MNISTLoader

import java.nio.file.{Files, Paths}

import org.platanios.tensorflow.api.core.{Shape, types

## Display MNIST Dataset

In [3]:
{{
def displayNumberMNIST(nb: Int) {
    val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
    val images = dataset.trainImages
    val imagesToDisplay = images.slice(0 :: nb, ::, ::)
    for (index <- 0 until nb) {
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imagesToDisplay(index).reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}
displayNumberMNIST(20)
}}

2019-09-11 09:22:31.627 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-09-11 09:22:33.114 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-09-11 09:22:33.119 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-09-11 09:22:33.179 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-09-11 09:22:33.180 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


In [3]:
val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
val trainImages = tf.data.datasetFromTensorSlices(dataset.trainImages.toFloat)
val trainLabels = tf.data.datasetFromTensorSlices(dataset.trainLabels.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(10000)
      .batch(256)
      .prefetch(10)

// Create the MLP model.
val input = Input(FLOAT32, Shape(-1, 28, 28))
val trainInput = Input(INT64, Shape(-1))
val layer = Flatten[Float]("Input/Flatten") >>
    Linear[Float]("Layer_0", 128) >> ReLU[Float]("Layer_0/Activation") >>
    Linear[Float]("Layer_1", 64) >> ReLU[Float]("Layer_1/Activation") >>
    Linear[Float]("Layer_2", 32) >> ReLU[Float]("Layer_2/Activation") >>
    Linear[Float]("OutputLayer", 10)
val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >>
    Mean("Loss/Mean")
val optimizer = tf.train.GradientDescent(1e-6f)
val model = Model.simpleSupervised(input, trainInput, layer, loss, optimizer)

// Create an estimator and train the model.
val estimator = InMemoryEstimator(model)
estimator.train(() => trainData, StopCriteria(maxSteps = Some(1000)))


2019-09-10 16:53:23.516 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-09-10 16:53:25.180 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-09-10 16:53:25.184 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-09-10 16:53:25.246 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-09-10 16:53:25.248 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


dataset: org.platanios.tensorflow.data.image.MNISTDataset = MNISTDataset(
  MNIST,
  Tensor[UByte, [60000, 28, 28]],
  Tensor[UByte, [60000]],
  Tensor[UByte, [10000, 28, 28]],
  Tensor[UByte, [10000]]
)
trainImages: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset]
trainLabels: ops.data.Dataset[Output[Long]] = Dataset[TensorSlicesDataset]
trainData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Zip/Repeat/Shuffle/Batch/Prefetch]
input: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@12c8cc33
trainInput: Input[Output[Long]] = org.platanios.tensorflow.api.learn.layers.Input@eae3211
layer: learn.layers.Compose[Output[Float], Output[Float], Output[Float]] = Compose(
  "Input/Flatten",
  Compose(
    "Input/Flatten",
    Compose(
      "Input/Flatten",
      Compose(
        "Input/Flatten",
        Compose(
          "Input/Flatten",
          Compose(
            "Input/Flatten",
            Compose(
              "Input

In [ ]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataSet.trainImages, dataSet.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataSet.testImages, dataSet.testLabels)}")

In [ ]:
/*def transformImagesForPrediction(imagefilePath: String, numChannel: Int, s: Session = Session()): Tensor[UByte] = {
    val tensorImageOuts = TImage.decodePng(Files.readFile(imagefilePath), numChannel)
    s.run(fetches = tf.reshape(tensorImageOuts, Shape(1,28,28)))
}

def runPrediction(imagePath: String): String = {
    val imageTest = transformImagesForPrediction(imagePath, 1)
    val imageToDisplay = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(tf.reshape(imageTest, Shape(28, 28, 1)))))
    Image(imageToDisplay.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    val predictions = estimator.infer(() => imageTest.toFloat)
    s"Le chiffre est : ${predictions.argmax(1).toInt.scalar}"
}*/
dataSet.trainImages.shape